# Declarations

<div class="alert alert-block alert-info">
    You can find all of the C programs in this notebook in the subdirectory containing this notebook:
    <code>./src/declarations</code>
</div>

> A declaration is a C language construct that introduces one or more identifiers into the program and specifies their meaning and properties.  https://en.cppreference.com/w/c/language/declarations

A declaration states that an identifier exists. An identifier must be declared before it is used in
an expression or statement so that the compiler can determine if the identifier is being used correctly.

## Variables

Variables must be declared before they can be used.
As in Java, a variable declaration starts with a type followed by a name:

```c
int x;
double y;

// declare multiple variables of the same type
int a, b, c;
```

Declaring a variable allocates memory for a value to be stored but does not initialize the value. The value
of an uninitialized variable is undefined:

In [ ]:
// var1.c

#include <stdio.h>

int main(void) {
    int x;
    double y;

    // declare multiple variables of the same type
    int a, b, c;
    
    printf("x = %d\n", x);
    printf("y = %f\n", y);
    printf("a = %d\n", a);
    printf("b = %d\n", b);
    printf("c = %d\n", c);
}

Variables may be initialized at their point of declaration. The term *definition* is used in the C language
specification to describe a declaration that also initializes the value of a variable:

In [ ]:
// var2.c

#include <stdio.h>

int main(void) {
    int x = 1;
    double y = -1.0;

    // declare multiple variables of the same type
    int a = -5, b = 3, c = 5;
    
    printf("x = %d\n", x);
    printf("y = %f\n", y);
    printf("a = %d\n", a);
    printf("b = %d\n", b);
    printf("c = %d\n", c);
}

#### `const` variables

The qualifier `const` may be used to indicate that a variable is read only. This is similar, but not exactly the
same, as a `final` variable in Java:

In [ ]:
// var3.c (does not compile)

#include <stdio.h>

int main(void) {
    const int x = 1;        // x is the constant 1
    
    const int y;
    y = 2;                  // oops, attempted assignment to const int
    
    printf("x = %d\n", x);
    printf("y = %d\n", y);
}

In C, the `const` becomes part of the type of the variable. In the above example, the types of `x` and `y` are
`const int`.

## Functions



A function declaration states the return type, name, and parameter list of a function. Function declarations
strongly resemble Java method headers (except that there are no access modifiers in C). The following function
declaration declares a function named `max2` that returns the maximum of its two parameters:

```c
double max2(double x, double y);
```

Note that the definition of `max2` is not given in the example above. However, if we declare a function in 
a compilation unit then we can
still use the function in the compilation unit. For example, we can write the following
source code file:

---
```c
// uses_max2.c 

#include <stdio.h>

// declares max2
double max2(double x, double y);

// defines max3
double max3(double x, double y, double z) {
	return max2(max2(x, y), z);
}

int main(void) {
	double a = 5.2;
	double b = -1.3;
	double c = 4.9;
	printf("max of %f, %f, %f = %f\n", a, b, c, max3(a, b, c));
}
```
---

On its own, `uses_max2.c` does not constitute a complete program because the definition of the function `max2`
is missing. We can still compile `uses_max2.c` to object code by using the `-c` option for `gcc`:

```sh
gcc -c uses_max2.c
```

The `-c` option instructs `gcc` to compile only. 

To create a complete program, the declared function must be defined somewhere, perhaps in a different
compilation unit. For example, we can create the following separate source code file:

---
```c
// max2.c

double max2(double x, double y) {
	return x >= y ? x : y;
}
```
---

and compile it to object code:

```sh
gcc -c max2.c
```

Finally, we can link the object code files into a single executable program:

```sh
gcc uses_max2.o max2.o -o uses_max2
```

Alternatively, we can compile all of source code files and link at once:

```sh
gcc max2.c uses_max2.c -o uses_max2
```

### Header files

Declaring functions defined in another file is error prone and may be impossible if the source code is of the other
file is not available. Instead of declaring the functions, a common mistake made by new C programmers is to
include the other file:

---
```c
// bad_uses_max2.c

#include <stdio.h>
#include "max2.c"              // DON'T DO THIS

// defines max3
double max3(double x, double y, double z) {
	return max2(max2(x, y), z);
}

int main(void) {
	double a = 5.2;
	double b = -1.3;
	double c = 4.9;
	printf("max of %f, %f, %f = %f\n", a, b, c, max3(a, b, c));
}
```
---

On first glance, this appears to work as intended. Compiling such a program works if done like so:

```sh
gcc bad_uses_max2.c -o bad_uses_max2
```

The first clue that something is amiss appears when attempting to compile the program like so:

```sh
gcc max2.c bad_uses_max2.c -o bad_uses_max2
```

or like so:

```sh
gcc -c max2.c
gcc -c bad_uses_max2.c
gcc max2.o bad_uses_max2.o -o bad_uses_max2
```

The previous two compilation attempts both produce a linker error complaining that `max2` has been
defined twice. This is because the definition of `max2` appears in the file `max2.c` and in the file
`bad_uses_max2.c` after `max2.c` has been included using `#include`. In C, the names of all functions
having *external linkage* must be unique across the entire program. By default, all functions have
external linkage unless they are marked as `static`.

The solution to this problem is to place the declaration of the function `max2` in a header file
and then `#include` the header file into any C source code file that uses the `max2` function. By convention,
the filename of a header file has the extension `.h`. The program in our example consists of three
files after creating the header file:

---
```c
#ifndef MAX2_H
#define MAX2_H

// max2.h

double max2(double x, double y);

#endif

```
---

---
```c
// max2.c

#include "max2.h"

double max2(double x, double y) {
	return x >= y ? x : y;
}

```
---

---
```c
// also_uses_max2.c

#include <stdio.h>
#include "max2.h"              // DO THIS

// defines max3
double max3(double x, double y, double z) {
	return max2(max2(x, y), z);
}

int main(void) {
	double a = 5.2;
	double b = -1.3;
	double c = 4.9;
	printf("max of %f, %f, %f = %f\n", a, b, c, max3(a, b, c));
}
```
---

A header file should always start with an *include guard* that defines an identifier for the header file.
In our example, the identifier is `MAX2_H` which uses the common convention of capitalizing the
filename of the header file and replacing the `.` with `_`. The preprocessor directive `#ifndef MAX2_H`
instructs the preprocessor to retain everything up to the closing `#endif` if the identifier `MAX2_H` 
*is not defined*. The first time the `max2.h` header file is included into a compilation unit,
the identifier `MAX2_H` will not be defined with causes the preprocessor to retain the following
lines of the header file:

```c
#define MAX2_H

// max2.h

double max2(double x, double y);
```

The preprocessor directive `#define MAX2_H` instructs the preprocessor to define the identifier
`MAX2_H`.

If the header file is included more than once into the same compilation unit, the existance of the
identifier `MAX2_H` will cause the directive `#ifndef MAX2_H` to skip over everything up to and
including the `#endif` which prevents multiple redeclarations of the function `max2`.